# Implementations of the Stochastic Gradient Hamilton Monte Carlo Algorithm

Authors: Tiancheng Pan, Lingyu Zhou and Fan Zhu

## Abstract

In this package, we followed and implemented the "Stochastic Gradient Hamiltonian Monte Carlo" algorithm, which was proposed by Tianqi Chen, Emily B. Fox and Carlos Guestrin (2014). To significantly reduce the computational complexity, this Hamilton Monte Carlo algorithm incorporates the stochastic gradient, which is computed on minibatchs of data with noise and counterbalances that noise by a friction term. The SGHMC algorithm was initially implemented in Python and then optimized through numba, Cython (C++) and multiprocessing. [Abstract on examples] The behaviors of SGHMC algorithm were examined on two examples compared to two other MCMC methods. [Briefly describe the tests] There is an up-to-date Github repository for our SGHMC codes at (https://github.com/TianchenPan/STA-663-Final-Project). The instructions of installation and explanation of this package are available in README file.

## Background

## Description of the Algorithm

## Describe Optimization for Performance

## Applications to Simulated Data Sets

## Applications to Real Data Sets

## Comparative Analysis with Competing Algorithms

## Discussion

## References